In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from sphere.assignment import sphere_assignment
from sphere.clustering import run_clustering

# ===== 配置 =====
patch_id = "patch_0_0.csv"
base_dir = "/home/wangzhuo/data/enact_synthetic_output/chunks"

bin_path = f"{base_dir}/bins_gdf/{patch_id}"
cell_path = f"{base_dir}/cells_gdf/{patch_id}"
gt_path   = f"{base_dir}/ground_truth_nuclei/{patch_id}"

# ===== 读取数据 =====
bin_df = pd.read_csv(bin_path)
bin_df["geometry"] = bin_df["geometry"].apply(wkt.loads)
bin_gdf = gpd.GeoDataFrame(bin_df, geometry="geometry")

cell_df = pd.read_csv(cell_path)
cell_df["geometry"] = cell_df["geometry"].apply(wkt.loads)
cell_gdf = gpd.GeoDataFrame(cell_df, geometry="geometry")

gt = pd.read_csv(gt_path).set_index("cell_id")
gt = gt.apply(pd.to_numeric, errors='coerce').fillna(0)

# ===== 公共基因 =====
exclude_cols = {'assigned_bin_id', 'row', 'column', 'geometry', 'cell_id'}
common_genes = sorted(list(set(bin_gdf.columns) & set(gt.columns) - exclude_cols))

print(f"公共基因数量: {len(common_genes)}")

# ===== 运行 SPHERE =====
pred_sphere = sphere_assignment(bin_gdf, cell_gdf, common_genes)

# ===== 聚类 =====
adata_clustered = run_clustering(pred_sphere)
print("聚类完成！")

公共基因数量: 485


/home/wangzhuo/miniconda3/envs/spatial_310/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:412: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(msg, UserWarning)
/home/wangzhuo/miniconda3/envs/spatial_310/lib/python3.10/site-packages/scanpy/preprocessing/_scale.py:316: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
2025-09-16 14:50:30.435354: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-16 14:50:30.441267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758005430.447454   399

聚类完成！
